# Business Understanding 

CMS rates hospitals in the US on a scale of 1-5 with the objective to make it easier for patients and consumers to compare the quality of hospitals.
The ratings directly influence the choice of the hospital made by consumers and may have a significant impact on the revenue earned by hospitals.
Thus, it is extremely important for hospitals to understand the methodology used by CMS for calculating the ratings so that they can work on
improving the factors that influence them.
 
This project is focused on developing an approach to calculate hospital ratings and using it to identify areas of improvement for
certain hospitals. It will also require a thorough understanding of the rating system developed by CMS.


# Business Problem 

The aim of analysis is to understand the methodology used by CMS for calculating the ratings and identify
the factors influencing the ratings for hospitals, so that they can work on improving the factors that influence them.
Recommend ways to improve the rating for Evanston Hospital to improve their current star rating of 3/5*



# Data Understanding #

The original source of data is `Hospital_Revised_FlatFiles_20161110`
  CSV Files
1.	Readmissions and Deaths - Hospital.csv	                                                readmission.csv
2.	Readmissions and Deaths - Hospital.csv +   Complications - Hospital.csv	                mortality.csv
3.	Healthcare Associated Infections - Hospital.csv +   Complications - Hospital.csv	      safety.csv
4.	HCAHPS - Hospital.csv	                                                                  experience.csv
5.	Outpatient Imaging Efficiency - Hopital.csv	                                            medical.csv
6.	Timely and Effective Care - Hospital.csv	                                              timeliness.csv
7.	Timely and Effective Care - Hospital.csv	                                              effectiveness.csv


# Exploratory Data Analysis 

- Perform the Univariate analysis for all the groups.
- Perform Bi-variate analysis for all the groups.


Modelling 

 Part 1 - Supervised Learning-Based Rating
 Part 2 - Factor analysis and Clustering-Based Rating (Unsupervised)
 Part 3 - Provider analysis - Recommendations for Hospitals

Let us load the data and create the groups as above:
Copied the required raw files to the Groups location.
1. "Readmissions and Deaths - Hospital.csv"
2. "Complications - Hospital.csv"
3. "Healthcare Associated Infections - Hospital.csv"
4. "HCAHPS - Hospital.csv"
5. "Outpatient Imaging Efficiency - Hospital.csv"
6. "Timely and Effective Care - Hospital.csv"



# Data Prepartion, cleaning and Supervised Modelling 
Data set contains 58 excel files, 2 PDF files out of this for this assignment we require 6 files & it has suffix as "_Hospital"

Load the files into dataframes:
1. Load the data - replace Not Available, Not Applicable with NA  (Suffix _Raw dataframes )
2. Split xxxx_rawdata frames into 2 data frames [xxxx_hosp, xxxx_meas)
3. Rename columns - Standardize names across
4. Reorder columns to match all files
5. Standardize the measures- some measures with positive zscores and some measures with negative zscores.
6. Impute the outliers

# Libraries to be used


In [5]:
import os
os.chdir("/Users/eklavya/projects/education/formalEducation/DataScience/DataScienceAssignments/HealthCare/Capstone/src/")


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_context('paper')



# Utility functions 

In [6]:
def print_ln():
    print('-' * 80, '\n')


# Function to create a subset of the dataframe
def subset(dataframe, col_name, col_names_list):
    return dataframe.loc[dataframe[col_name].isin(col_names_list)]

# Converts the datatype of a specific column and returns the new dataframe
def func_numeric(df, col_name):
    df[col_name] = df[col_name].astype(float)
    return df

# Renames a column in the dataframe by appending `_score`.
def func_rename(df):
    old_col_names = df.columns.to_list()
    new_col_names = []
    for a_col_name in old_col_names:
        col_name = a_col_name + "_score"
        new_col_names.append(col_name)

    name_pairs = dict(zip(old_col_names, new_col_names))
    df = df.rename(columns=name_pairs)
    return df

# Function to compute the negative zscore value for the dataframe

def negative_zscore(dataframe):
    df = dataframe.copy()
    cols = list(df.columns)
    for col in cols:
        df[col] = - (df[col] - df[col].mean())/df[col].std(ddof=0)
    return df

# Function to compute the positive zscore value for the dataframe
def positive_zscore(dataframe):
    df = dataframe.copy()
    cols = list(df.columns)
    for col in cols:
        df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)
    return df

# Function to compute the valid subset of a dataframe i.e. reduces outliers via IQR method
def subset_by_iqr(df, column, whisker_width=0):
    # Calculate Q1, Q2 and IQR
    q1 = df[column].quantile(0.00125)
    q3 = df[column].quantile(0.99875)
    iqr = q3 - q1
    # Apply filter with respect to IQR, including optional whiskers
    filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
    return df.loc[filter][column]


# Driver function for `subset_by_iqr` which treats the outliers in a dataframe
def treat_outliers(dataframe):
    df = dataframe.copy()
    cols = list(df.columns)
    for col in cols:
        df[col] = subset_by_iqr(df, col)
    return df

In [8]:

# Computes the group score of a dataframe after cleaning and doing PCA on the group of measures
def function_group_score(numeric_df, score_name):
    # CMS recommends atleast 3 non-null measures per group
    df = numeric_df.dropna(thresh= 3)
    imputed_df = df.apply(lambda x: x.fillna(x.median()), axis=0)
    pca = IncrementalPCA()
    df_pca = pca.fit_transform(imputed_df)
    df_pca = pd.DataFrame(df_pca, columns= df.columns)
    df_pca.index = df.index
    df_with_weight = df_pca.mean(axis=1)
    df_scores = pd.DataFrame({score_name : df_with_weight})
    return df_scores

## 1. Readmission - Load "Readmissions and Deaths - Hospital.csv" file into read_raw

In [9]:
# Readmissions and deaths 

read_rawdata = pd.read_csv("Readmissions and Deaths - Hospital.csv",
                           encoding="ISO-8859-1",
                           na_values=["Not Available", "Not Applicable"])

## Declaring the list of important measures associated with this particular group
read_meas_list =   ["READM_30_AMI", "READM_30_CABG", "READM_30_COPD", "READM_30_HF", "READM_30_HIP_KNEE", "READM_30_HOSP_WIDE", "READM_30_PN", "READM_30_STK"]

read_hosp = read_rawdata.iloc[:,0:8]


In [10]:
read_hosp = read_hosp.drop_duplicates(keep='first')

In [ ]:
read_meas = read_rawdata.iloc[: , [0,9,12]]

In [ ]:
# Converting the datatype of Score to a float

read_meas['Score'] = read_meas['Score'].astype(float)

